In [12]:
import anndata as ad
from pathlib import Path
import numpy as np

In [2]:
comp_set = ad.read_h5ad("/raid/kreid/v_cell/competition_support_set/competition_train.h5")

In [3]:
de_labels = comp_set.uns["de_labels"]

In [13]:
for key in de_labels.keys():
    print(f"Key: {key}, Value: {np.count_nonzero(de_labels[key])}" if isinstance(de_labels[key], np.ndarray) else f"Key: {key}, Value: {de_labels[key]}")

Key: ARC_H1_ACAT2, Value: 6215
Key: ARC_H1_ACVR1B, Value: 2057
Key: ARC_H1_AKT2, Value: 1328
Key: ARC_H1_ANTXR1, Value: 215
Key: ARC_H1_ARID1A, Value: 5276
Key: ARC_H1_ATM, Value: 4669
Key: ARC_H1_ATP1B1, Value: 174
Key: ARC_H1_ATP6V0B, Value: 518
Key: ARC_H1_ATP6V0C, Value: 573
Key: ARC_H1_BIRC2, Value: 5611
Key: ARC_H1_BRD9, Value: 1090
Key: ARC_H1_C1QBP, Value: 5147
Key: ARC_H1_CALM3, Value: 2309
Key: ARC_H1_CAMSAP2, Value: 215
Key: ARC_H1_CASP2, Value: 593
Key: ARC_H1_CASP3, Value: 7316
Key: ARC_H1_CAST, Value: 4434
Key: ARC_H1_CDCA2, Value: 1566
Key: ARC_H1_CENPO, Value: 2447
Key: ARC_H1_CHMP3, Value: 3444
Key: ARC_H1_CLDN6, Value: 580
Key: ARC_H1_CLDN7, Value: 1270
Key: ARC_H1_CREG1, Value: 286
Key: ARC_H1_DAXX, Value: 2266
Key: ARC_H1_DHCR24, Value: 1129
Key: ARC_H1_DHX36, Value: 10060
Key: ARC_H1_DLG5, Value: 2029
Key: ARC_H1_DNAJA3, Value: 2582
Key: ARC_H1_DNMT1, Value: 9439
Key: ARC_H1_DZIP3, Value: 150
Key: ARC_H1_EID2, Value: 342
Key: ARC_H1_EIF4B, Value: 5807
Key: ARC_H1_E

In [2]:
import numpy as np
import pandas as pd
from anndata import AnnData
from pdex import parallel_differential_expression
from tqdm import tqdm


def compute_de_labels(adata: AnnData,
                           perturb_col: str = "target_gene",
                           cell_type_col: str = "cell_type",
                           control_var: str = "non-targeting",
                           alpha: float = 0.05) -> None:
    """
    Precompute DE gene labels (+1, -1, 0) using pdex.
    Stores results in adata.uns['de_labels'].

    Parameters
    ----------
    adata : AnnData
        Input AnnData object.
    perturb_col : str
        Column in obs indicating perturbation/target gene.
    cell_type_col : str
        Column in obs indicating cell type.
    control_var : str
        Name of the control group in perturb_col.
    alpha : float
        FDR threshold for significance.
    """

    cell_types = adata.obs[cell_type_col].unique()
    target_genes = adata.obs[perturb_col].unique()

    result = {}

    # iterate over cell types
    for ct in tqdm(cell_types, desc="Computing DE labels with pdex"):
        # subset AnnData to current cell type
        adata_ct = adata[adata.obs[cell_type_col] == ct].copy()
        
        # Run pdex differential expression
        de_df = parallel_differential_expression(
            adata_ct,
            reference=control_var,
            groupby_key=perturb_col,
            metric="wilcoxon",  # default
            num_workers=20,
            batch_size=100,
        )

        # if "fdr" < 0.05 and "percent_change" > 0 set to 1, if "fdr" < 0.05 and "percent_change" < 0 set to -1, else 0
        de_df["de_embedding"] = de_df.apply(
            lambda row: 1 if row["fdr"] < alpha and row["percent_change"] > 0 else (-1 if row["fdr"] < alpha and row["percent_change"] < 0 else 0),
            axis=1
        )
        
        for target in target_genes:
            # filter for current target gene
            de_target = de_df[de_df["target"] == target]
            if not de_target.empty:
                result[f"{ct}_{target}"] = de_target["de_embedding"].values
            else:
                result[f"{ct}_{target}"] = 0

    adata.uns['de_labels'] = result
    print(f"Stored DE results in adata.uns['de_labels'] with {len(adata.uns['de_labels'])} entries.")
    


In [3]:
import numpy as np
import pandas as pd
from anndata import AnnData
from tqdm import tqdm

def compute_expression_ranks(
    adata: AnnData,
    groupby_key: str = "target_gene",
    celltype_key: str = "cell_type",
) -> None:
    """
    Precompute gene expression ranks for each (cell_type, target_gene).
    Stores results in adata.uns['expr_ranks'].

    Each entry is a vector of length n_vars where
        rank[i] = rank of gene i's mean expression within that subset.
    """
    results = {}

    cell_types = adata.obs[celltype_key].unique()
    target_genes = adata.obs[groupby_key].unique()

    # single progress bar over all combinations
    combos = [(ct, tg) for ct in cell_types for tg in target_genes]
    for ct, tg in tqdm(combos, desc="Computing expression ranks", total=len(combos)):
        # subset to matching cells
        mask = (adata.obs[celltype_key] == ct) & (adata.obs[groupby_key] == tg)
        if mask.sum() == 0:
            continue

        subset = adata[mask]

        # mean expression across cells for each gene
        mean_expr = np.asarray(subset.X.mean(axis=0)).ravel()

        # compute ranks (highest expression = rank 1)
        ranks = pd.Series(mean_expr).rank(method="first", ascending=False).to_numpy()

        results[f"{ct}_{tg}"] = ranks

    adata.uns["rank_embedding"] = results
    print(f"Stored rank vectors in adata.uns['rank_embedding'] with {len(results)} entries.")


In [10]:
data_dir = Path("/raid/kreid/v_cell/competition_support_set")

In [11]:
anndata_paths = [file for file in data_dir.glob("*.h5")]

In [12]:
for path in anndata_paths:
    print(f"Processing {path.name}")
    adata = ad.read_h5ad(path)

    # Compute DE labels
    compute_de_labels(adata, perturb_col="target_gene", cell_type_col="cell_type", control_var="non-targeting", alpha=0.05)

    # Compute expression ranks
    compute_expression_ranks(adata, groupby_key="target_gene", celltype_key="cell_type")

    # Save updated AnnData
    adata.write_h5ad(path)  # Overwrite original file or save to a new path if needed

Processing hepg2.h5


Identifying target masks: 100%|██████████| 69/69 [00:00<00:00, 50044.44it/s]
INFO:pdex._single_cell:Precomputing variable indices for each feature
Identifying variable indices: 100%|██████████| 18080/18080 [00:00<00:00, 8518649.33it/s]
INFO:pdex._single_cell:Creating shared memory memory matrix for parallel computing
INFO:pdex._single_cell:Creating generator of all combinations: N=1247520
INFO:pdex._single_cell:Creating generator of all batches: N=12476
INFO:pdex._single_cell:Initializing parallel processing pool
INFO:pdex._single_cell:Processing batches
Processing batches: 100%|██████████| 12476/12476 [00:39<00:00, 318.27it/s]
INFO:pdex._single_cell:Flattening results
INFO:pdex._single_cell:Closing shared memory pool
Computing DE labels with pdex: 100%|██████████| 1/1 [00:44<00:00, 44.71s/it]


Stored DE results in adata.uns['de_labels'] with 69 entries.


Computing expression ranks: 100%|██████████| 69/69 [00:00<00:00, 377.24it/s]


Stored rank vectors in adata.uns['rank_embedding'] with 69 entries.
Processing k562.h5


...

Stored rank vectors in adata.uns['rank_embedding'] with 69 entries.
Processing rpe1.h5


Identifying target masks: 100%|██████████| 69/69 [00:00<00:00, 44897.14it/s]
INFO:pdex._single_cell:Precomputing variable indices for each feature
Identifying variable indices: 100%|██████████| 18080/18080 [00:00<00:00, 4839993.38it/s]
INFO:pdex._single_cell:Creating shared memory memory matrix for parallel computing
INFO:pdex._single_cell:Creating generator of all combinations: N=1247520
INFO:pdex._single_cell:Creating generator of all batches: N=12476
INFO:pdex._single_cell:Initializing parallel processing pool
INFO:pdex._single_cell:Processing batches
Processing batches: 100%|██████████| 12476/12476 [01:18<00:00, 158.08it/s]
INFO:pdex._single_cell:Flattening results
INFO:pdex._single_cell:Closing shared memory pool
Computing DE labels with pdex: 100%|██████████| 1/1 [01:25<00:00, 85.50s/it]


Stored DE results in adata.uns['de_labels'] with 69 entries.


Computing expression ranks: 100%|██████████| 69/69 [00:00<00:00, 236.53it/s]


Stored rank vectors in adata.uns['rank_embedding'] with 69 entries.
